In [1]:
import os
import time
import requests
import pandas as pd
import yfinance as yf

from selenium import webdriver

In [2]:
# Chrome Driver options

curdir = os.getcwd()

chrome_options = webdriver.ChromeOptions()

prefs = {'download.default_directory' : curdir}
chrome_options.add_experimental_option('prefs', prefs)

In [3]:
# Download IDIV index composition from B3

B3_URL = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IDIV'

driver = webdriver.Chrome('./chromedriver', options=chrome_options)

def download_idiv(url):

    driver.get(url)

    download_button = driver.find_element_by_link_text('Download')

    download_button.click()

    time.sleep(1)

download_idiv(B3_URL)    
    
driver.close()

In [104]:
# Obter o código das ações que compõe o IDIV

idiv_list = [file for file in os.listdir(curdir) if str.startswith(file, 'IDIV')]

path = f'{curdir}/{idiv_list[0]}'

df = pd.read_csv(path, sep=';', header=1, skipfooter=2, encoding='latin-1', engine='python')

lista_codigos = list(df.index.values)

print(lista_codigos)

['ABCB4', 'AESB3', 'ALUP11', 'B3SA3', 'BRSR6', 'BBSE3', 'BBDC3', 'BBDC4', 'BRAP4', 'BBAS3', 'AGRO3', 'CMIG3', 'CMIG4', 'CSMG3', 'CPLE3', 'CPLE6', 'CPFE3', 'CYRE3', 'DIRR3', 'ELET3', 'ELET6', 'ENAT3', 'ENBR3', 'EGIE3', 'FESA4', 'GRND3', 'MYPK3', 'ITSA4', 'JHSF3', 'LEVE3', 'BEEF3', 'PETR3', 'PSSA3', 'QUAL3', 'RAPT4', 'ROMI3', 'SANB11', 'CSNA3', 'SYNE3', 'TAEE11', 'TGMA3', 'VIVT3', 'TRPL4', 'TRIS3', 'UNIP6', 'VALE3', 'WIZS3']


In [143]:
# Delete downloaded .csv file

for file in os.listdir(curdir):
    
    if str.endswith(file, '.csv'):
        
        os.remove(file)
        
    else:
        
        pass

In [108]:
# Status Invest

REQUEST_BASE_URL = 'https://statusinvest.com.br/acao/companytickerprovents?ticker={}&chartProventsType=2'

In [137]:
def dividend_info(ativos: list) -> pd.DataFrame:

    df = pd.DataFrame()

    for codigo in ativos:

        res = requests.get(REQUEST_BASE_URL.format(codigo))

        d = dict(res.json())

        data = pd.DataFrame(d['assetEarningsYearlyModels'])

        data = data.assign(stock=codigo)

        data = data.rename(
            columns={'rank': 'year', 'value': 'dividend_per_share'})

        data = data[['stock', 'year', 'dividend_per_share']]  # reorder columns

        df = pd.concat([df, data])

    return df


df = dividend_info(ativos=lista_codigos)

df_dividend = df.groupby('stock')[['dividend_per_share']].mean()

df_dividend

,dividend_per_share
stock,
ABCB4,0.715218
AESB3,0.169167
AGRO3,0.922144
ALUP11,0.960276
B3SA3,0.419177
BBAS3,1.648842
BBDC3,0.924224
BBDC4,1.016647
BBSE3,1.704368


In [136]:
# Obter as cotações atualizadas para calcular o Dividend Yield

lista_cotacoes = []

for codigo in lista_codigos:
    
    ticker = yf.Ticker(codigo + '.SA')
    
    c = ticker.info['regularMarketPrice']
    
    lista_cotacoes.append(c)

[19.01,
 9.93,
 28.96,
 12.47,
 11.16,
 28.47,
 16.19,
 19.52,
 22.88,
 40.64,
 26.44,
 19.35,
 12.75,
 13.42,
 6.54,
 7.14,
 36.22,
 14.19,
 11.83,
 48.16,
 49.8,
 15.61,
 23.67,
 41.82,
 50.37,
 7.38,
 15.37,
 9.31,
 6.44,
 24.65,
 14.51,
 36.81,
 20.51,
 10.85,
 11.02,
 13.42,
 30.45,
 15.82,
 4.6,
 41.9,
 16.15,
 44.38,
 24.34,
 4.64,
 92.67,
 66.66,
 8.64]

In [142]:
df_market_price = pd.DataFrame({'stock': lista_codigos, 'market_price': lista_cotacoes})

df_dividend2 = df_dividend.merge(df_market_price, on='stock')

df_dividend2['dividend_yield'] = (df_dividend2['dividend_per_share']/df_dividend2['market_price'])*100

df_dividend2

,stock,dividend_per_share,market_price,dividend_yield
0,ABCB4,0.715218,19.01,3.762326
1,AESB3,0.169167,9.93,1.703595
2,AGRO3,0.922144,26.44,3.487685
3,ALUP11,0.960276,28.96,3.315871
4,B3SA3,0.419177,12.47,3.361481
5,BBAS3,1.648842,40.64,4.057191
6,BBDC3,0.924224,16.19,5.708611
7,BBDC4,1.016647,19.52,5.208230
8,BBSE3,1.704368,28.47,5.986541
9,BEEF3,0.404122,14.51,2.785130
